# Exam result processing

- Copy exam roster csv file (one used for exam check-in tool to cs220_tools ex: exam2_roster.csv)
- Download exam result csv and move it to current directory ex: exam2_results.csv
- Download canvas roster and save it sa canvas.csv in current directory ex: canvas.csv
- Update below section on dropping question(s) if needed
- Writes new csv canvas_exam_name.csv ex: canvas_exam2.csv

In [1]:
import sys
import json
import boto3
import csv

In [2]:
exam_name = input("Enter exam name: exam1 or exam2 or final: ")

drop_flag = False
# KEY: exam version; VALUE: list of questions to drop / regrade
drop_questions = {
        "6": {"_10": "1"},
        "7": {"_20": "1"},
        "8": {"_10": "1"},
        "9": {"_25": "1"},
        }

roster_dict = {}
backup_roster = {}
netid_roster_dict = {}

BUCKET = 'caraza-harter-cs301'
s3 = boto3.client('s3')

Enter exam name: exam1 or exam2 or final: exam1


In [3]:
exam_roster_file = exam_name + "_roster.csv"
exam_results_file = exam_name + "_results.csv"

In [4]:
# inspired by https://automatetheboringstuff.com/2e/chapter16/
def process_csv(filename):
    example_file = open(filename, encoding="utf-8")
    example_reader = csv.reader(example_file)
    example_data = list(example_reader)
    example_file.close()
    return example_data

In [5]:
def convert_roster_to_netid_roster():
    for student_id in roster_dict:
        net_id = roster_dict[student_id]["netid"]
        roster_dict[student_id]["campus id"] = student_id
        netid_roster_dict[net_id] = roster_dict[student_id]

In [6]:
def read_roster(exam_roster):
    exam_roster_header = exam_roster[0]
    exam_roster = exam_roster[1:]
    for row in exam_roster:
        campus_id = row[exam_roster_header.index("id")]
        netid = row[exam_roster_header.index("login")]
        lname = row[exam_roster_header.index("lname")].lower().strip()
        fname = row[exam_roster_header.index("fname")].lower().strip()
        full_name = lname + "," + fname
        roster_dict[campus_id] = {
                "netid": netid,
                "lname": lname,
                "fname": fname
                }
        backup_roster[full_name] = {
                "campus id": campus_id,
                "netid": netid,
                "lname": lname,
                "fname": fname
        }

In [7]:
def gen_email(exam_name):
    emails = []
    for student_id in roster_dict:
        email = {
            "html":True, 
            "subject": "CS 220 - " + exam_name.upper() + " results", 
            "to": roster_dict[student_id]["netid"] + "@wisc.edu"
        }
        if "total score" not in roster_dict[student_id]:
            #print(email["to"])
            msg = """<p>Please see canvas for your exam score and answer choices, 
                    if you took exam via canvas quiz.</p>"""
            msg += """<p>If you took an in-person exam, do not worry that you are seeing this message. 
                    Please email your instructor.</p>"""
        else:
            #print(student_id, roster_dict[student_id])
            msg = '<p>Hello, your exam score was: ' + roster_dict[student_id]["total score"]
            msg += "<p>Your answers for each question:</p>"
            msg += "<ol>"
            for qnum in range(len(roster_dict[student_id]["answers"])):
                msg += "<li>" + roster_dict[student_id]["answers"][qnum] + "</li>"
            msg += "</ol>"
        
        email["message"] = msg
        emails.append(email)

    with open(exam_name + "_emails.json", "w") as f:
        json.dump(emails, f)

In [8]:
def update_canvas_score(exam_name, exam_column, exam_honorlock_column):
    canvas_data = process_csv("canvas.csv")
    header = canvas_data[0]
    canvas_data = canvas_data[1:]
    for row in canvas_data:
        if row[header.index(exam_honorlock_column)] != "N/A":
            row[header.index(exam_column)] = row[header.index(exam_honorlock_column)]
        else:
            net_id = row[header.index("SIS Login ID")].lower()
            net_id = net_id.replace("@wisc.edu", "")
            if net_id in netid_roster_dict:
                if "total score" in netid_roster_dict[net_id]:
                    row[header.index(exam_column)] = netid_roster_dict[net_id]["total score"]
                else:
                    if net_id in netid_roster_dict:
                        print(netid_roster_dict[net_id])
                    else:
                        print("Couldn't find:", net_id)
                    row[header.index(exam_column)] = 0
            else:
                if net_id in netid_roster_dict:
                    print(netid_roster_dict[net_id])
                else:
                    print("Couldn't find:", net_id)
                row[header.index(exam_column)] = 0

    canvas_data.insert(0, header)

    with open("canvas_" + exam_name + ".csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(canvas_data)

In [9]:
def read_exam_results(exam_results):
    exam_results_header = exam_results[0]
    exam_results = exam_results[1:]
    for row in exam_results:
        lname = row[exam_results_header.index("LastName")].lower().strip()
        fname = row[exam_results_header.index("FirstName")].lower().strip()
        campus_id = row[exam_results_header.index("ID")]
        special_codes = row[exam_results_header.index("SpecialCodes")].strip()
        codes = special_codes.split(" ")
        cleaned_codes = [code for code in codes if code != ""]
        if len(cleaned_codes) > 1:
            exam_version = cleaned_codes[1]
        else:
            exam_version = cleaned_codes[0]

        full_name = lname + "," + fname
        if campus_id not in roster_dict:
            # Initial check to filter out incorrect campus ID entries on the scantron
            # There will inevitably be some incorrect entries!
            if full_name in backup_roster:
                # Print this and send these students an email :)
                print("Wrong campus ID: ", end = "")
                print(backup_roster[full_name]["netid"] + "@wisc.edu")
                campus_id = backup_roster[full_name]["campus id"]
                roster_dict[campus_id] = backup_roster[full_name]
            else:
                # Good luck with the manual finding process ;)
                print("Couldn't find student: ", end = "")
                print(row)
                #pass

        # Checks for questions which needs to be dropped
        # Gives points to students who chose different answer from
        # original answer option
        if drop_flag:
            drop = drop_questions[exam_version]
            for question in drop:
                student_answer = row[exam_results_header.index(question)]
                if student_answer != drop[question]:
                    roster_dict[campus_id]["total score"] = str(int(row[exam_results_header.index("TotalScore")]) + 1)
                else:
                    roster_dict[campus_id]["total score"] = row[exam_results_header.index("TotalScore")]
        else:
            #If all mappings work, this will populate dictionary with total score
            roster_dict[campus_id]["total score"] = row[exam_results_header.index("TotalScore")]

        #Marked answers from scantron
        answers = []
        for col_idx in range(7, 42):
            answers.append(row[col_idx])

        roster_dict[campus_id]["answers"] = answers

In [10]:
exam_roster = process_csv(exam_roster_file)
read_roster(exam_roster)

exam_results = process_csv(exam_results_file)
read_exam_results(exam_results)

Wrong campus ID: das38@wisc.edu
Wrong campus ID: iluitel@wisc.edu
Wrong campus ID: oh79@wisc.edu
Wrong campus ID: sbreidigan@wisc.edu
Wrong campus ID: ewermeling@wisc.edu
Wrong campus ID: jberget@wisc.edu
Wrong campus ID: ndjefferson@wisc.edu


In [11]:
gen_email(exam_name)
convert_roster_to_netid_roster()

In [12]:
exam_column = input("Enter exam canvas column: ")
exam_honorlock_column = input("Enter exam canvas honorlock column: ")
update_canvas_score(exam_name, exam_column, exam_honorlock_column)

Enter exam canvas column: Exam 1 (1469410)
Enter exam canvas honorlock column: Exam 1 Honorlock (1517837)
{'netid': 'cmbowe', 'lname': 'bowe', 'fname': 'connor', 'campus id': '9082397432'}
{'netid': 'zguo274', 'lname': 'guo wang', 'fname': 'zecheng', 'campus id': '9083540881'}
{'netid': 'oqhaas', 'lname': 'haas', 'fname': 'olivia', 'campus id': '9083130949'}
{'netid': 'ytang232', 'lname': 'tang', 'fname': 'yuxin', 'campus id': '9081647308'}
Couldn't find: 2603481baa70f937120319518be1e0f9abd9d34f
